In [ ]:
!pip install tk
!pip install genanki
!mkdir anki
!ls

In [ ]:
! gdown --id 1iXsQwXlIFFlil1zzMF-WaKWVV_KZCyml
! unzip cards.zip

Downloading...
From: https://drive.google.com/uc?id=1iXsQwXlIFFlil1zzMF-WaKWVV_KZCyml
To: /content/cards.zip
100% 106M/106M [00:01<00:00, 80.5MB/s]
Archive:  cards.zip
  inflating: 1133/1270/88d56449-de7d-49e3-b723-3b630253860e.png  
  inflating: 1133/1270/03c65ae4-4f6c-46c4-b0be-80314ce15911.png  
  inflating: 1133/1270/2d7a42c3-07e3-4b6d-a3d5-ad9b6c9eacd4.png  
  inflating: 1133/1270/user_1270.json  
  inflating: 1133/1270/fe31e26b-c916-4b7c-943b-7614da95fcda.png  
  inflating: 1133/1270/096cb17d-8ba4-478d-8d73-bc8fd5502dcb.png  
  inflating: 1133/1270/5d1b56c9-209c-471b-95ee-8601887b83a0.png  
  inflating: 1133/1270/85bb77e8-cf83-4e6d-94dd-8fc786ec1979.png  
  inflating: 1133/1270/1270.json     
  inflating: 1133/1270/571cd682-5f33-42bc-8b9f-188d155419f2.png  
  inflating: 1133/1270/70bc1c75-6982-41b1-a7ef-1b13f0ca6b8b.png  
  inflating: 1133/1270/035e5941-a528-464f-af50-197883dc7a50.png  
  inflating: 1133/1270/580d55e4-7256-4cab-a6be-0ab588505969.png  
  inflating: 1133/1270/22995

## Model.py

In [ ]:
from copy import copy
from cached_property import cached_property
import chevron
import yaml

class Model:

  FRONT_BACK = 0
  CLOZE = 1
  DEFAULT_LATEX_PRE = ('\\documentclass[12pt]{article}\n\\special{papersize=3in,5in}\n\\usepackage[utf8]{inputenc}\n'
                       + '\\usepackage{amssymb,amsmath}\n\\pagestyle{empty}\n\\setlength{\\parindent}{0in}\n'
                       + '\\begin{document}\n')
  DEFAULT_LATEX_POST = '\\end{document}'

  def __init__(self, model_id=None, name=None, fields=None, templates=None, css='', model_type=FRONT_BACK,
               latex_pre=DEFAULT_LATEX_PRE, latex_post=DEFAULT_LATEX_POST, sort_field_index=0):
    self.model_id = model_id
    self.name = name
    self.set_fields(fields)
    self.set_templates(templates)
    self.css = css
    self.model_type = model_type
    self.latex_pre = latex_pre
    self.latex_post = latex_post
    self.sort_field_index = sort_field_index

  def set_fields(self, fields):
    if isinstance(fields, list):
      self.fields = fields
    elif isinstance(fields, str):
      self.fields = yaml.full_load(fields)

  def set_templates(self, templates):
    if isinstance(templates, list):
      self.templates = templates
    elif isinstance(templates, str):
      self.templates = yaml.full_load(templates)

  @cached_property
  def _req(self):
    """
    List of required fields for each template. Format is [tmpl_idx, "all"|"any", [req_field_1, req_field_2, ...]].
    Partial reimplementation of req computing logic from Anki. We use chevron instead of Anki's custom mustache
    implementation.
    The goal is to figure out which fields are "required", i.e. if they are missing then the front side of the note
    doesn't contain any meaningful content.
    """
    sentinel = 'SeNtInEl'
    field_names = [field['name'] for field in self.fields]

    req = []
    for template_ord, template in enumerate(self.templates):
      required_fields = []
      for field_ord, field in enumerate(field_names):
        field_values = {field: sentinel for field in field_names}
        field_values[field] = ''

        rendered = chevron.render(template['qfmt'], field_values)

        if sentinel not in rendered:
          # when this field is missing, there is no meaningful content (no field values) in the question, so this field
          # is required
          required_fields.append(field_ord)

      if required_fields:
        req.append([template_ord, 'all', required_fields])
        continue

      # there are no required fields, so an "all" is not appropriate, switch to checking for "any"
      for field_ord, field in enumerate(field_names):
        field_values = {field: '' for field in field_names}
        field_values[field] = sentinel

        rendered = chevron.render(template['qfmt'], field_values)

        if sentinel in rendered:
          # when this field is present, there is meaningful content in the question
          required_fields.append(field_ord)

      if not required_fields:
        raise Exception(
          'Could not compute required fields for this template; please check the formatting of "qfmt": {}'.format(
            template))

      req.append([template_ord, 'any', required_fields])

    return req

  def to_json(self, timestamp: float, deck_id):
    for ord_, tmpl in enumerate(self.templates):
      tmpl['ord'] = ord_
      tmpl.setdefault('bafmt', '')
      tmpl.setdefault('bqfmt', '')
      tmpl.setdefault('bfont', '')
      tmpl.setdefault('bsize', 0)
      tmpl.setdefault('did', None)  # TODO None works just fine here, but should it be deck_id?

    for ord_, field in enumerate(self.fields):
      field['ord'] = ord_
      field.setdefault('font', 'Liberation Sans')
      field.setdefault('media', [])
      field.setdefault('rtl', False)
      field.setdefault('size', 20)
      field.setdefault('sticky', False)

    return {
      "css": self.css,
      "did": deck_id,
      "flds": self.fields,
      "id": str(self.model_id),
      "latexPost": self.latex_post,
      "latexPre": self.latex_pre,
      "latexsvg": False,
      "mod": int(timestamp),
      "name": self.name,
      "req": self._req,
      "sortf": self.sort_field_index,
      "tags": [],
      "tmpls": self.templates,
      "type": self.model_type,
      "usn": -1,
      "vers": []
    }

  def __repr__(self):
    attrs = ['model_id', 'name', 'fields', 'templates', 'css', 'model_type']
    pieces = ['{}={}'.format(attr, repr(getattr(self, attr))) for attr in attrs]
    return '{}({})'.format(self.__class__.__name__, ', '.join(pieces))

## Making Basic card - taken from builtin_models.py

In [ ]:
import random
import genanki
from datetime import datetime
import json
import re
import os

#model_id = random.randrange(1 << 30, 1 << 31)

BASIC_MODEL = Model(
  1559383000,
  'Basic (genanki)',
  fields=[
    {
      'name': 'Question',
      'font': 'Arial',
    },
    {
      'name': 'Back',
      'font': 'Arial',
    },
    {'name': 'MyMedia'}
  ],
  templates=[
    {
      'name': 'Card 1',
      'qfmt': '{{Question}}<br><br>{{MyMedia}}',
      'afmt': '{{FrontSide}}\n\n<hr id=answer>\n\n{{Back}}',
    },
  ],
  css='.card {\n font-family: arial;\n font-size: 20px;\n text-align: center;\n color: black;\n background-color: white;\n}\n}',
)

def make_card(card):
  image_path = card["image_path"].split('/')[-1]
  myMedia = f'<img height="500px" src="{image_path}">'
  return genanki.Note(model=BASIC_MODEL,fields=[card['question'], card['answer'], myMedia])


def make_deck(deck_name, cards):
  deck_id = random.randrange(1 << 30, 1 << 31)
  my_deck = genanki.Deck(deck_id, deck_name)

  for card in cards:
    my_deck.add_note(make_card(card))

  date = datetime.now()
  anki_questions_file = "/content/anki" + f"{date}.apkg"

  my_package = genanki.Package(my_deck)
  image_paths = [ card['image_path'] for card in cards] 
  my_package.media_files = image_paths 
  my_package.write_to_file(anki_questions_file)


#returns list
def iterate_over_deck_json_file(folder_path):
  folder_name = folder_path.split('/')[-2]
  path_to_json_file = folder_path + folder_name + '.json'
  cards = []
  with open(path_to_json_file) as json_file:
      deck_file = json.load(json_file)
  for card in deck_file['cards']:
    #need to add relative path to this
    image_path = folder_path + card["image"].split('/')[-1]
    cards.append({'question':card["frontTextBox"]["text"],'answer':card["correctAnswer"],'image_path':image_path})
  return cards


deck_path = '/content/1133'
with open('/content/1133/1133.json') as deck_file:
    deck_file = json.load(deck_file)

sub_deck_names = {}
for sub_deck in deck_file['decks']:
  sub_deck_id = sub_deck['id']
  sub_deck_names[sub_deck_id] = 'CC: Anatomy and Physiology::' + sub_deck['title']

for root, dirs, files in os.walk(deck_path):
  if dirs:
    #iterate over directories in outer folder
    for dir in dirs:
      #make deck for inner folder
      sub_deck_name = sub_deck_names[dir]
      deck_folder_path = deck_path + '/' + dir + '/' 
      #print(deck_folder_path)
      make_deck(sub_deck_name, iterate_over_deck_json_file(deck_folder_path))

#Testing - need to be in this format!
# cards = [
#         {
#           'question':'1',
#           'answer':'1',
#           'image_path':'/content/Any cats?.jpeg'
#         },
#         {
#           'question':'2',
#           'answer':'2',
#           'image_path':'/content/Any cats?.jpeg'
#         },
#         {
#           'question':'3',
#           'answer':'3',
#           'image_path':'/content/Any cats?.jpeg'
#         }   
# ]

# make_deck('Test dict', cards)


To do
• Iterate through deck directory and make cards for each

* fill_dict (need to update image path
* iterate over folder



In [ ]:
deck_folder_path
folder_name = deck_folder_path.split('/')[-2]
path_to_json_file = deck_folder_path + folder_name + '.json'
print(deck_folder_path)
print(path_to_json_file)

/content/1133/1261/
/content/1133/1261/1261.json


In [ ]:
deck_path = '/content/1133'
with open('/content/1133/1133.json') as deck_file:
    deck_file = json.load(deck_file)

sub_deck_names = {}
for sub_deck in deck_file['decks']:
  sub_deck_id = sub_deck['id']
  sub_deck_names[sub_deck_id] = 'CC: Anatomy and Physiology::' + sub_deck['title']

In [ ]:
print(len(sub_deck_names))
print(type(sub_deck_names))
print(sub_deck_names)

48
<class 'dict'>
{'1168': 'CC: Anatomy and Physiology::Anatomy and Physiology Preview', '1232': 'CC: Anatomy and Physiology::Intro to Anatomy and Physiology', '1233': 'CC: Anatomy and Physiology::The Fabric of Your Body', '1234': 'CC: Anatomy and Physiology::Epithelial Tissue', '1235': 'CC: Anatomy and Physiology::Intro to Connective Tissues', '1236': 'CC: Anatomy and Physiology::Types of Connective Tissues', '1237': 'CC: Anatomy and Physiology::Skin Deep ', '1238': 'CC: Anatomy and Physiology::Skin Deeper', '1239': 'CC: Anatomy and Physiology::Intro to the Nervous System', '1240': 'CC: Anatomy and Physiology::Action! Potential!', '1241': 'CC: Anatomy and Physiology::Synapses', '1242': 'CC: Anatomy and Physiology::Central Nervous System', '1243': 'CC: Anatomy and Physiology::Pain and Your Peripheral Nervous System', '1244': 'CC: Anatomy and Physiology::Intro to the Autonomic Nervous System', '1245': 'CC: Anatomy and Physiology::Physiology of Stress: Your Sympathetic Nervous System ', 

In [ ]:
for key,value in sorted(sub_deck_names.items()):
  print(value)

CC: Anatomy and Physiology::Anatomy and Physiology Preview
CC: Anatomy and Physiology::Intro to Anatomy and Physiology
CC: Anatomy and Physiology::The Fabric of Your Body
CC: Anatomy and Physiology::Epithelial Tissue
CC: Anatomy and Physiology::Intro to Connective Tissues
CC: Anatomy and Physiology::Types of Connective Tissues
CC: Anatomy and Physiology::Skin Deep 
CC: Anatomy and Physiology::Skin Deeper
CC: Anatomy and Physiology::Intro to the Nervous System
CC: Anatomy and Physiology::Action! Potential!
CC: Anatomy and Physiology::Synapses
CC: Anatomy and Physiology::Central Nervous System
CC: Anatomy and Physiology::Pain and Your Peripheral Nervous System
CC: Anatomy and Physiology::Intro to the Autonomic Nervous System
CC: Anatomy and Physiology::Physiology of Stress: Your Sympathetic Nervous System 
CC: Anatomy and Physiology::The Parasympathetic Nervous System
CC: Anatomy and Physiology::Taste and Smell
CC: Anatomy and Physiology::Hearing and Balance
CC: Anatomy and Physiology::V